Handle all the imports 

In [ ]:
# We are at: 
# ACTIVE/fbpic/LASYFB/NewWavefronts.ipynb

import sys
import os
from importlib import reload
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

sys.path.insert(0, '../../../CNN/Old/ML/src/generation/') 
import utility_fnc
reload(utility_fnc)
from train import *
from ssim import compute_ssim_numpy
from criterion import *

import dataset as ds

Load the experimental spots

In [ ]:

# sys.path.insert(0, '../../../CNN/Old/ML/src/generation/')
trn_dir='../../../CNN/Old/ML/src/generation/'
data_dir = trn_dir+'Real3/'
# data_dir = trn_dir+'temp_2/'

#     resize, normalize_to
SetSz = len(os.listdir(data_dir))
print(SetSz)
dataset_size = 200 #SetSz #loading real dataset
dataset = ds.psf_dataset(root_dir = data_dir, 
                      size = dataset_size,
                      transform = transforms.Compose([ToTensor(), utility_fnc.resize()])) #Normalize(), 

Reload Models


In [ ]:
# Load model architectures and weights
import json
from Flex import U1
from U1D import U1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device='cpu'

model_dir =trn_dir+'mod_PIL_total_/modelfinal.pth' # model path
state_dict = torch.load(model_dir, map_location='cpu')
new_state_dict = OrderedDict()

from U1D_dropout import U1_drop

dropout=0.012 # from 0.02 (put in value used here)
dropout_bottleneck=0.02 # from 0.07 (put in value used here)
model = U1_drop(1, 1, dropout_enc=dropout_bottleneck,dropout_bottleneck=dropout_bottleneck,dropout_dec=dropout_bottleneck).to(device)

# model = U1(1, 1) #.to(device)
for k, v in state_dict.items():
    name = k[7:] # remove module.
    new_state_dict[name] = v
model.load_state_dict(state_dict)

Testing on old Data Set


In [6]:
 
import torch.nn as nn    
import utility_fnc
reload(utility_fnc)

idds=np.arange(0, 50)

device='cpu'
lasers=[]
phs=[]
dif=[]
rms=[]
MSE=[]
mse=nn.MSELoss()
SSIM=[]
mnMx=[]
for id_ in idds:
            d=dataset[id_]
            img = torch.as_tensor(d['image']) #.unsqueeze(0)d['phase']
            print(img)
            print(np.shape(img), print(img.max()))
#             utility_fnc.plotter(img)
            vmax=img.max()
            vmin=img.min()
            
            norm1 = (img - torch.min(img)) / (torch.max(img) - torch.min(img))
#             utility_fnc.plotter(norm1)
            output = model(norm1.unsqueeze(0).unsqueeze(0))
            phase_est = output[0].detach().to('cpu').numpy()
            phs.append(phase_est)
            phase=0.8*phase_est/(2*np.pi)
            dif.append(np.max(phase)-np.min(phase))
            rms.append(np.sqrt(np.mean((phase*(0.8*1e-6)/(2*np.pi))**2)))
            print(1e6*np.max(phase*(0.8*1e-6)/(2*np.pi)))
            print(1e6*np.min(phase*(0.8*1e-6)/(2*np.pi)))
            print('rms', rms[-1])
            mnMx.append(1e6*np.max(phase*(0.8*1e-6)/(2*np.pi))-1e6*np.min(phase*(0.8*1e-6)/(2*np.pi)))
            
            
            I_pred, las_obj=utility_fnc.propagator(phase_est[0], f0=9)
            img2=I_pred.squeeze(0).squeeze(0).detach().cpu().numpy()**2
            norm = (img2 - np.min(img2)) / (np.max(img2) - np.min(img2))
            utility_fnc.plotter(norm) #, vmin=vmin, vmax=vmax)
            utility_fnc.plotter(phase.squeeze(0))   
            lasers.append(las_obj)
            print(np.shape(norm), np.shape(norm1))
            npNorm=norm
            pdNorm=np.pad(npNorm, ((3, 3), (3, 3)))
            plt.imshow(pdNorm)
            plt.show()
            norm1=utility_fnc.pad_and_resize(norm1, np.shape(pdNorm))
            img=utility_fnc.pad_and_resize(img, np.shape(norm))
            plt.show()
            img[img==0]==1
            print(img.squeeze(0).squeeze(0))
#             utility_fnc.plotter(np.sqrt(img.squeeze(0).squeeze(0))) #, vmin=vmin, vmax=vmax)
#             utility_fnc.plotter(np.sqrt(img2))
#             norm = (img2 - np.min(img2)) / (np.max(img2) - np.min(img2))
            #norm1=np.pad(norm1, pad_width=((),()))
            MSE.append(mse(torch.as_tensor(norm1), torch.as_tensor(pdNorm)))
            print(np.shape(norm1), np.shape(norm))
#             npNorm=norm
#             pdNorm=np.pad(npNorm, ((10, 10), (10, 10)))
#             plt.imshow(pdNorm)
#             plt.show()
            
            norm1=utility_fnc.pad_and_resize(norm1, np.shape(pdNorm))
            SSIM.append(compute_ssim_numpy(norm1.squeeze().squeeze().numpy(), pdNorm))
            print(SSIM[-1])


/var/tmp/pbs.1816038.pbs/ipykernel_2291810/4247516163.py:18: ResourceWarning: unclosed file <_io.BufferedReader name='../../../CNN/Old/ML/src/generation/Real3/psf_0.fits'>
  d=dataset[id_]


tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -4.2625e+14,
         -2.4357e+14,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.0446e+14,
          7.9160e+14,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -3.6536e+14,
          6.0893e+13,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
tensor(3.9580e+15)
torch.Size([256, 280]) None
0.12294664486489637
-0.3196923614723346
rms 9.106549e-08
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor(5.4

/var/tmp/pbs.1816038.pbs/ipykernel_2291810/4247516163.py:18: ResourceWarning: unclosed file <_io.BufferedReader name='../../../CNN/Old/ML/src/generation/Real3/psf_1.fits'>
  d=dataset[id_]


0.10499531555296926
-0.29529670086958504
rms 8.441644e-08
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.5165e+14,
         -1.4066e+14, -2.8132e+14],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -7.0330e+13,
         -6.3297e+14,  7.0330e+13],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -2.1099e+14,
         -3.5165e+14,  7.0330e+14],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
tensor(8.3833e+16)
torch.Size([256, 256]) None


/var/tmp/pbs.1816038.pbs/ipykernel_2291810/4247516163.py:18: ResourceWarning: unclosed file <_io.BufferedReader name='../../../CNN/Old/ML/src/generation/Real3/psf_2.fits'>
  d=dataset[id_]


...

0.0646345057475628
-0.3106209476300137
rms 8.9822585e-08
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -4.0010e+15,
          1.0669e+16, -7.3352e+15],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -6.6684e+14,
          1.0003e+16, -2.6674e+15],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  6.6684e+14,
         -9.3358e+15,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
tensor(3.6376e+18)
torch.Size([256, 256]) None


/var/tmp/pbs.1816038.pbs/ipykernel_2291810/4247516163.py:18: ResourceWarning: unclosed file <_io.BufferedReader name='../../../CNN/Old/ML/src/generation/Real3/psf_48.fits'>
  d=dataset[id_]


0.06906930849481796
-0.30417066909649293
rms 8.826068e-08
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.2266e+14,
          2.2586e+15,  1.9360e+15],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -3.2266e+14,
         -5.1626e+15, -3.8719e+15],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  9.6798e+14,
          2.5813e+15, -9.6798e+14],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
tensor(1.6098e+18)
torch.Size([256, 256]) None


/var/tmp/pbs.1816038.pbs/ipykernel_2291810/4247516163.py:18: ResourceWarning: unclosed file <_io.BufferedReader name='../../../CNN/Old/ML/src/generation/Real3/psf_49.fits'>
  d=dataset[id_]


0.07298035598068964
-0.34137019611080177
rms 9.079419e-08
